In [ ]:
def append_to_sys_path(folder: str, base_folder: str = "weather_bundle") -> str:
    """
    Append the specified folder to the system path if it is not already present.
    Allows importing packages from the folder.
    """
    import re
    import os
    import sys

    if not (match := re.search(rf"(.+[\\/]{base_folder})", os.getcwd())):
        raise FileNotFoundError(f"Could not find {base_folder} in current path: {os.getcwd()}")

    base_path = match.group(1)
    for root, dirs, _ in os.walk(base_path):
        if "src" in dirs:
            base_path = root
            break

    if (path := os.path.join(base_path, folder)) not in sys.path:
        sys.path.append(path)
    return path


append_to_sys_path("src/config")

In [ ]:
from config import Config

config = Config()
catalog = config["weather"]["catalog_name"]
bronze_schema = config["weather"]["bronze_schema_name"]
noaa: dict = config["weather"]["noaa"]
file_status_table_name = noaa["metrics"]["bronze_file_status_table_name"]

table_name = f"{catalog}.{bronze_schema}.{file_status_table_name}"

print(f"{table_name=}")

In [ ]:
# Remove all records from the table
spark.sql(f"DELETE FROM {table_name}")

In [ ]:
# Print number of records after deletion
count = spark.sql(f"SELECT COUNT(*) as cnt FROM {table_name}").collect()[0]["cnt"]
print(f"Number of records after deletion: {count}")